In [5]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import matplotlib.pyplot as plt

%matplotlib inline 

# 0. データセットの読み込み

In [6]:
HR_DATASET_PATH = '../datasets/HR_comma_sep.csv'
hr_df = pd.read_csv(HR_DATASET_PATH)
hr_df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [7]:
print(hr_df.shape)
print(hr_df.columns)
print(hr_df.isnull().any())

(14999, 10)
Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'sales', 'salary'],
      dtype='object')
satisfaction_level       False
last_evaluation          False
number_project           False
average_montly_hours     False
time_spend_company       False
Work_accident            False
left                     False
promotion_last_5years    False
sales                    False
salary                   False
dtype: bool


In [8]:
# salary(給与水準)をダミー変数へ置換する
hr_df.salary.replace({'low': 1, 'medium': 2, 'high': 3}, inplace=True)
# salesをダミー変数へ
hr_df = pd.get_dummies(hr_df, columns=['sales'])

# サポートベクトルマシン(SVM: Support Vector Machine)
*SVMは線形二値分類器であり、非常に高い分類性能を持つことで知られている。また、後に説明するカーネル法を組み合わせて用いれば、SVMは非線形な分類も可能である。*

 - **正クラス(positive class)**
 - **負クラス(negative class)**
 - **正例**(positive example, positive instance)
 - **負例**(negative example, negative instance)

訓練データ$D$は、
$$
D=\left\{ \left( x^{ (1) },y^{ (1) } \right) ,\left( x^{ (2) },y^{ (2) } \right) ,\cdots ,\left( x^{ (\left| D \right| ) },y^{ (\left| D \right| ) } \right)  \right\} 
$$
と与えられているとしよう。

 - $x^{(i)}$: 特徴ベクトル
 - $y^{ (i) }=\begin{cases} 正例のクラスラベル:\quad +1 \\ 負例のクラスラベル:\quad -1 \end{cases}$

SVMは線形分類器であるので、分離平面の方向ベクトル$w$と切片$b$をパラメータとして、
$$
f\left(x\right) = w \cdot x - b
$$
という関数を用いて表される。特徴ベクトル$x$を、$f\left(x\right) \ge 0$ならば正クラス、$f\left(x\right) < 0$ならば負クラスに分類する。

 - [scikit-learn 0.19.0 documentation - sklearn.svm: Support Vector Machines](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm)

## サポートベクトルマシンの導出
> $d$次元の訓練データが分布しているとして、これらを正クラス、負クラスに分類する平面**分離平面(separating plane)**を構築したい。

![](http://www.cis.doshisha.ac.jp/mjin/R/31/fig04.PNG)

最適な分離平面は、
$$
w \cdot x - b = 0\\
w \cdot x = b
$$
を満たす点$x$の集合になる。これを$x_{\ast}$とする。このときのパラメータ$w$と$b$を求める。

マージン最大の最適識別超平面は
$$
\min { \quad \frac { 1 }{ 2 } { w }^{ 2 } } \\ s.t.\quad { y }^{ \left( i \right)  }\left( w\cdot { x }^{ \left( i \right)  }-b \right) -1\ge 0;\forall i
$$
で定義され、この不等式制約付き最適化問題は、主問題、双対問題を解くことで得られる。

 - ※目的関数に$1/2$がついているが、これはあってもなくても同じであることに注意。あえて付けたのは、単にその後の計算がわかりやすくなるからである。

## 線形分離可能でない場合への拡張



## 非線形SVMの導出



In [5]:
from sklearn.svm import LinearSVC, SVC # 線形SVM, 非線形SVM
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

use_cols = [
    'satisfaction_level',
    'last_evaluation',
    'number_project',
    'average_montly_hours',
    'time_spend_company',
    'Work_accident',
    'promotion_last_5years',
    'salary',
    'sales_IT',
    'sales_RandD',
    'sales_accounting',
    'sales_hr',
    'sales_management', 
    'sales_marketing',
    'sales_product_mng',
    'sales_sales',
    'sales_support', 
    'sales_technical'
]

# 離職者数:在籍者数 = 1:1に直す
X1 = hr_df[hr_df.left == 1][use_cols]
X0 = hr_df[hr_df.left == 0][use_cols].sample(len(X1))
X = pd.concat([X1, X0])
Y = hr_df.loc[X.index, 'left']

# 標準化
transformed_cols = [
    'satisfaction_level',
    'last_evaluation',
    'number_project',
    'average_montly_hours',
    'time_spend_company',
]
ss = StandardScaler()
ss.fit(X[transformed_cols])
X[transformed_cols] = ss.transform(X[transformed_cols])


# 交差検証(ホールドアウト法)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, train_size=0.7, random_state=0)


for model in [LinearSVC, SVC]:
    linear_svc = model()
    linear_svc.fit(x_train, y_train)

    print('学習データ数: %s' % x_train.shape[0])
    print('学習データのうち離職した人数: %s' % y_train[y_train == 1].shape[0])
    print('検証データ数: %s' % x_test.shape[0])
    print('検証データのうち離職した人数: %s' % y_test[y_test == 1].shape[0])

    y_pred = linear_svc.predict(x_test)

    print('---モデルの評価---')
    print('正確度: %s' % accuracy_score(y_test, y_pred))
    print('適合率: %s' % precision_score(y_test, y_pred))
    print('再現率: %s' % recall_score(y_test, y_pred))
    print('F値: %s' % f1_score(y_test, y_pred))

    print('---分割表---')
    confusion_df = pd.DataFrame(confusion_matrix(y_test, y_pred), index=['在籍者', '離職者'], columns=['在籍すると予測', '離職すると予測'])
    print(confusion_df)
    print('--------------------------------------------------------------')

/Users/taiyou/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


学習データ数: 4999
学習データのうち離職した人数: 2510
検証データ数: 2143
検証データのうち離職した人数: 1061
---モデルの評価---
正確度: 0.762949136724
適合率: 0.752511415525
再現率: 0.776625824694
F値: 0.764378478664
---分割表---
     在籍すると予測  離職すると予測
在籍者      811      271
離職者      237      824
--------------------------------------------------------------
学習データ数: 4999
学習データのうち離職した人数: 2510
検証データ数: 2143
検証データのうち離職した人数: 1061
---モデルの評価---
正確度: 0.934204386374
適合率: 0.942307692308
再現率: 0.923656927427
F値: 0.932889100428
---分割表---
     在籍すると予測  離職すると予測
在籍者     1022       60
離職者       81      980
--------------------------------------------------------------


In [6]:
from sklearn.svm import LinearSVC, SVC # 線形SVM, 非線形SVM
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

use_cols = [
    'satisfaction_level',
    'last_evaluation',
    'number_project',
    'average_montly_hours',
    'time_spend_company',
    'Work_accident',
    'promotion_last_5years',
    'salary',
    'sales_IT',
    'sales_RandD',
    'sales_accounting',
    'sales_hr',
    'sales_management', 
    'sales_marketing',
    'sales_product_mng',
    'sales_sales',
    'sales_support', 
    'sales_technical'
]

# 離職者数:在籍者数 = 1:1に直す
X1 = hr_df[hr_df.left == 1][use_cols]
X0 = hr_df[hr_df.left == 0][use_cols].sample(len(X1))
X = pd.concat([X1, X0])
Y = hr_df.loc[X.index, 'left']

# 標準化
transformed_cols = [
    'satisfaction_level',
    'last_evaluation',
    'number_project',
    'average_montly_hours',
    'time_spend_company',
]
ss = StandardScaler()
ss.fit(X[transformed_cols])
X[transformed_cols] = ss.transform(X[transformed_cols])

# 交差検証(k-分割交差検証法)
CV = 10
for model in [LinearSVC, SVC]:
    y_pred = cross_val_predict(model(), X, Y, cv=CV)

    print('正確度: %s' % accuracy_score(Y, y_pred))
    print('適合率: %s' % precision_score(Y, y_pred))
    print('再現率: %s' % recall_score(Y, y_pred))
    print('F値: %s' % f1_score(Y, y_pred))
    print('分割表')
    confusion_df = pd.DataFrame(confusion_matrix(Y, y_pred), index=['在籍者', '離職者'], columns=['在籍すると予測', '離職すると予測'])
    print(confusion_df)
    print('----------------------------------------------------------------------------')

正確度: 0.768272192663
適合率: 0.753976670201
再現率: 0.796415569868
F値: 0.774615279858
分割表
     在籍すると予測  離職すると予測
在籍者     2643      928
離職者      727     2844
----------------------------------------------------------------------------
正確度: 0.934192103052
適合率: 0.943887775551
再現率: 0.923270792495
F値: 0.933465458664
分割表
     在籍すると予測  離職すると予測
在籍者     3375      196
離職者      274     3297
----------------------------------------------------------------------------


In [ ]:
from sklearn.svm import LinearSVC, SVC # 線形SVM, 非線形SVM
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

# Grid searchの詳細
# http://qiita.com/SE96UoC5AfUt7uY/items/c81f7cea72a44a7bfd3a
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

use_cols = [
    'satisfaction_level',
    'last_evaluation',
    'number_project',
    'average_montly_hours',
    'time_spend_company',
    'Work_accident',
    'promotion_last_5years',
    'salary',
    'sales_IT',
    'sales_RandD',
    'sales_accounting',
    'sales_hr',
    'sales_management', 
    'sales_marketing',
    'sales_product_mng',
    'sales_sales',
    'sales_support', 
    'sales_technical'
]

# アンダーサンプリング(離職者数:在籍者数 = 1:1に直す)
X1 = hr_df[hr_df.left == 1][use_cols]
X0 = hr_df[hr_df.left == 0][use_cols].sample(len(X1))
X = pd.concat([X1, X0])
Y = hr_df.loc[X.index, 'left']

# 標準化
transformed_cols = [
    'satisfaction_level',
    'last_evaluation',
    'number_project',
    'average_montly_hours',
    'time_spend_company',
]
ss = StandardScaler()
ss.fit(X[transformed_cols])
X[transformed_cols] = ss.transform(X[transformed_cols])

# Grid searchのパラメータ
parameters = [
        {'C': [1, 10, 100, 1000], 'kernel': ['linear']}, # 線形カーネル
        {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.001, 0.0001]}, # RBFカーネル
        {'C': [1, 10, 100, 1000], 'kernel': ['poly'], 'degree': [2, 3, 4], 'gamma': [0.001, 0.0001]}, # 多項式カーネル
        {'C': [1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.001, 0.0001]} # シグモイドカーネル
    ]

SCORE = 'f1'
CV = 10

clf = GridSearchCV(
        SVC(), # 識別器
        parameters, # 最適化したいパラメータセット 
        cv=CV, # 交差検定の回数
        scoring='%s_weighted' % SCORE # モデルの評価関数の指定
    )
clf.fit(X, Y)

In [ ]:
print(clf.grid_scores_)
print(clf.best_params_)
print(clf.best_estimator_)

In [ ]:
y_pred = cross_val_predict(clf.best_estimator_, X, Y, cv=CV)

print('正確度: %s' % accuracy_score(Y, y_pred))
print('適合率: %s' % precision_score(Y, y_pred))
print('再現率: %s' % recall_score(Y, y_pred))
print('F値: %s' % f1_score(Y, y_pred))
print('分割表')
confusion_df = pd.DataFrame(confusion_matrix(Y, y_pred), index=['在籍者', '離職者'], columns=['在籍すると予測', '離職すると予測'])
confusion_df